<a href="https://colab.research.google.com/github/celsosoaresss/TCC/blob/main/TCC_Emotion_Class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **IMPORTAÇÃO DE BIBLIOTECAS NECESSÁRIAS**

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab.patches import cv2_imshow
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten, BatchNormalization

from google.colab import drive
drive.mount('/content/drive')

# **PRÉ-PROCESSAMENTO**

In [ ]:
#BASE DE TREINO
gerador_treinamento = ImageDataGenerator(rescale=1./255,
                                         rotation_range=7,
                                         horizontal_flip=True,
                                         zoom_range=0.2)

dataset_treinamento = gerador_treinamento.flow_from_directory('/content/drive/MyDrive/emotion_class/images/images/train',
                                                              target_size = (48, 48),
                                                              batch_size = 16,
                                                              class_mode = 'categorical',
                                                              shuffle = True)

#BASE DE TESTE
gerador_teste = ImageDataGenerator(rescale=1./255)


# **CONTRUÇÃO E TREINAMENTO DA CNN**

In [ ]:
numero_detectores = 32
numero_classes = 7
largura, altura = 48, 48
epocas = 70

network = Sequential()
network.add(Conv2D(filters=numero_detectores, kernel_size=(3,3), activation='relu', padding='same', input_shape=(largura, altura, 3)))
network.add(BatchNormalization())
network.add(Conv2D(filters=numero_detectores, kernel_size=(3,3), activation='relu', padding='same'))
network.add(BatchNormalization())
network.add(MaxPooling2D(pool_size=(2,2)))
network.add(Dropout(0.2))

network.add(Conv2D(filters=2*numero_detectores, kernel_size=(3,3), activation='relu', padding='same'))
network.add(BatchNormalization())
network.add(Conv2D(filters=2*numero_detectores, kernel_size=(3,3), activation='relu', padding='same'))
network.add(BatchNormalization())
network.add(MaxPooling2D(pool_size=(2,2)))
network.add(Dropout(0.2))

network.add(Conv2D(filters=2*2*numero_detectores, kernel_size=(3,3), activation='relu', padding='same'))
network.add(BatchNormalization())
network.add(Conv2D(filters=2*2*numero_detectores, kernel_size=(3,3), activation='relu', padding='same'))
network.add(BatchNormalization())
network.add(MaxPooling2D(pool_size=(2,2)))
network.add(Dropout(0.2))

network.add(Conv2D(filters=2*2*2*numero_detectores, kernel_size=(3,3), activation='relu', padding='same'))
network.add(BatchNormalization())
network.add(Conv2D(filters=2*2*2*numero_detectores, kernel_size=(3,3), activation='relu', padding='same'))
network.add(BatchNormalization())
network.add(MaxPooling2D(pool_size=(2,2)))
network.add(Dropout(0.2))


network.add(Flatten())


network.add(Dense(units=2 * numero_detectores, activation='relu'))
network.add(BatchNormalization())
network.add(Dropout(0.2))


network.add(Dense(units=2 * numero_detectores, activation='relu'))
network.add(BatchNormalization())
network.add(Dropout(0.2))

network.add(Dense(units=numero_classes, activation='softmax'))
print(network.summary())

In [ ]:
network.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
#network.fit(dataset_treinamento, epochs=epocas)

In [ ]:
with open('/content/drive/MyDrive/network_emotions/network_emotions.json', 'r') as json_file:
  json_saved_model = json_file.read()
json_saved_model

In [ ]:
network_loaded = tf.keras.models.model_from_json(json_saved_model)
network_loaded.load_weights('/content/drive/MyDrive/network_emotions/weights_emotions.hdf5')
network_loaded.compile(loss = 'categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

In [ ]:
network_loaded.summary()

# **AVALIAÇÃO DA REDE NEURAL**

In [ ]:
network_loaded.evaluate(dataset_teste)

In [ ]:
previsoes = network_loaded.predict(dataset_teste)
previsoes

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(dataset_teste.classes, previsoes)

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(dataset_teste.classes, previsoes)
cm

In [ ]:
sns.heatmap(cm, annot=True);

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(dataset_teste.classes, previsoes))

# **CLASSIFICAÇÃO DE IMAGEM FORA DO CONJUNTO DE TREINO E TESTE**

In [ ]:
imagem = cv2.imread('/content/drive/MyDrive/fotos para teste/WhatsApp Image 2023-03-08 at 11.12.18.jpeg')
cv2_imshow(imagem)

In [ ]:
detector_face = cv2.CascadeClassifier('/content/drive/MyDrive/fotos para teste/haarcascade_frontalface_default.xml')

In [ ]:
imagem_original = imagem.copy()
deteccoes = detector_face.detectMultiScale(imagem_original)

In [ ]:
roi = imagem[258:258 + 288,  236:236 + 288]
cv2_imshow(roi)

In [ ]:
roi = cv2.resize(roi, (48,48))
cv2_imshow(roi)

In [ ]:
probs = network_loaded.predict(roi)
probs

In [ ]:
previsao = np.argmax(probs)
previsao

# **CLASSIFICAÇÃO EM VÍDEO**

In [ ]:
cap = cv2.VideoCapture('/content/drive/MyDrive/fotos para teste/IMG_3133.MOV')
conectado, video = cap.read()
print(conectado, video.shape)

In [ ]:
save_path = '/content/drive/MyDrive/fotos para teste/IMG_3133_result.avi'
fourcc = cv2.VideoWriter_fourcc(*'XVID')
fps = 24
output_video = cv2.VideoWriter(save_path, fourcc, fps, (video.shape[1], video.shape[0]))

In [ ]:
emotions = ['Angry', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Sad', 'Surprise']

In [ ]:
while (cv2.waitKey(1) < 0):
  conectado, frame = cap.read()
  if not conectado:
    break
  deteccoes = detector_face.detectMultiScale(frame, scaleFactor=1.2,minNeighbors=5, minSize=(30,30))
  if len(deteccoes) > 0:
    for (x, y, w, h) in deteccoes:
      frame = cv2.rectangle(frame, (x, y), (x + w, y + h), (0,255,0), 2)
      roi = frame[y:y + h, x:x + w]
      roi = cv2.resize(roi, (48,48))
      roi = roi / 255
      roi = np.expand_dims(roi, axis = 0)
      previsao = network_loaded.predict(roi)

      if previsao is not None:
        resultado = np.argmax(previsao)
        cv2.putText(frame, emotions[resultado], (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0), 1, cv2.LINE_AA)

  cv2_imshow(frame)
  output_video.write(frame)

print('Terminou!')
output_video.release()
cv2.destroyAllWindows()